## Problem set 2: Naive Bayes classification

Answer each question. You may discuss answers with other students, but write your own solution. Some questions have more than one part. Most questions involve coding, some require you to comment.

Can you tell whether a review of a restaurant is positive or negative? What words are most indicative? We'll examine data from Yelp, provided as part of the Yelp Academic Challenge.

To do this, we will create a program (a classifier) that uses actual examples of one-star and five-star reviews to classify a new review as positive or negative.  This classifier will estimate the probability that a review is negative or positive based on the words in that review and assign the most likely label (negative or positive).

Some terminology: the word "word" can mean two different things, so we will be more specific. The phrase "red fish blue fish" contains a sequence of four word *tokens*, but only three distinct word *types* (red, fish, blue). 

### Q1: guesses before looking at data

Before we start coding, list five word types that you think stongly indicate a positive review and five that indicate a negative review:

Positive words here: amazing, awesome, delicious, nice, recommend
Negative words here: bad, average, bad, slow, rude

Now we will create a program that uses actual examples of one-star and five-star reviews to classify text as positive and negative.

We start by partioning our collection of pre-labeled restaurant reviews into two sets, positive reviews and negative reviews. (We have done this for you.) We will then count the relative frequency of each distinct word type in the two sub-collections which will allow us to estimate the probability that a given word appears in a review given its polarity (positive or negative). These individual word probabilites will then be multiplied to etimate the probability of multiple word tokens appearing in a document given its polarity. Finally, we can then appply Bayes theorem to switch our conditionals: i.e. to estimate the probability a review is positive or negative based on the words in it.

Here's the math. The first line is an application of Bayes' rule. The second line expands the marginal probability of the words into the sum over positive and negative polarities. The third line adds the "naive" assumption that words are independent, as long as we know the polarity. (Note that I flipped the order of the prior probability and the word probability in the third line, to make it clear where the product ends.)

\begin{align}
P(positive|w_1, w_2, w_3, ..., w_N) & = \frac{P(w_1, w_2, w_3, ..., w_N | positive)P(positive)}{P(w_1, w_2, w_3, ..., w_N)} \\
&= \frac{P(w_1, w_2, w_3, ..., w_N | positive)P(positive)}{P(w_1, w_2, w_3, ..., w_N | positive)P(positive) + P(w_1, w_2, w_3, ..., w_N | negative)P(negative)} \\
&\approx \frac{P(positive)\prod_i P(w_i|positive)}{P(positive)\prod_i P(w_i|positive) + P(negative)\prod_i P(w_i|negative) }
\end{align}

### Q2: create a pattern

The first step is to count the words in each of the two *training* sub-collections. They are in two files, `positive.txt` and `negative.txt`. Each line in these files is one review. Before we start, we need to decide how we are going to break reviews into words.

Create a regular expression that matches sequences of one or more letter characters. Then write a short sentence and use the `findall` function to list all of the substrings of that sentence that match your pattern. Include at least one common word in your sentence that will not be handled well by this pattern.

In [19]:
import re
my_reg_exp=re.compile("\w+")
my_reg_exp.findall("This class rocks!")

['This', 'class', 'rocks']

### Q3: Specify variables to store data

To apply Bayes' rule to estimate the probability of polarity given a review, we need the probability of words given polarity and the probability of positive or negative polarity.

First create variables that will store this information.
For the positive reviews create a `Counter` that will store the counts of words in positive reviews. Also create a variable that will count the number of positive reviews. Create similar variables for the negative reviews.

In [20]:
from collections import Counter
pos_word_counts = Counter()
neg_word_counts = Counter()
num_pos_reviews = 0
num_neg_reviews = 0

##Counter has a function .update list

### Q4: Read data from files

Now read the two files, collecting the count of each distinct word in each sub-collection, as well as the number of reviews in each sub-collection. You will need to *update* the appropriate `Counter` for each review.

Once you have processed the two files, test the counters by printing the five most common words in each counter.

In [21]:
import re
##re.compile(my_reg_exp) will create a regular expression object define by my_reg_exp. 
##.findall(string) will return all substrings in string matching the regular expression object
with open("positive.txt", "r") as pos_reader:
    try:
        for review in pos_reader:
            ##my_reg_exp = re.compile('[a-z]+')
            ##q = my_reg_exp.findall(line)
            ##print (q)
            pos_word_counts.update(my_reg_exp.findall(review))
            num_pos_reviews += 1
    except UnicodeDecodeError:
        pass    
with open("negative.txt", "r") as neg_reader:
    try:
        for review in neg_reader:
            ##my_reg_exp = re.compile('[a-z]+')
            ##q = my_reg_exp.findall(line)
            ##print (q)
            neg_word_counts.update(my_reg_exp.findall(review))
            num_neg_reviews += 1
    except UnicodeDecodeError:
        pass           
print(num_pos_reviews)
print(num_neg_reviews)

temp_list = []
for key in pos_word_counts:
    temp_list.append(pos_word_counts[key])

        
##print("positive reviews:" + str(pos_word_counts))
##print("negative reviews:" + str(neg_word_counts))

##PRINT MOST COMMON WORDS


3883
790


### Q5: prior probability

Calculate the probability that a randomly selected review is positive. 

In [22]:
all_reviews = num_pos_reviews + num_neg_reviews 
prob_pos = num_pos_reviews/all_reviews
print (prob_pos)

0.8309437192381768


Does this value surprise you? Why or why not?

This value surprises me because from my own experience reading reviews (outside of Yelp), people only bother to leave reviews if they have something negative to say. The fact that 80.26% of reviews are positive says that Yelp users seem to love being active on the site and leaving reviews even when they have don't have a bad experience! Very nice of them!

### Q6: count word totals

Create two variables, `positive_tokens` and `negative_tokens`, whose value is the total number of word tokens in that sub-collection. Print these values.

In [23]:

positive_tokens = 0
for key in pos_word_counts:
    positive_tokens = positive_tokens + pos_word_counts[key]
print(positive_tokens)

negative_tokens = 0
for key in neg_word_counts:
    negative_tokens = negative_tokens + neg_word_counts[key]
print(negative_tokens)

##positive_tokens = sum(pos_word_counts.values())
##negative_tokens = sum(neg_word_counts.values())

##print(positive_tokens)
##print(negative_tokens)

415765
126358


### Q7: calculate word probability (version 1.0)

Create a function `word_prob` that takes three arguments: a word, a counter, and a sum. It should return the probability of the word estimated from that counter. Assume that the sum is accurate, you don't need to check for errors.

Use this function to print the probability of the words "delicious", "manager", "edgy", and "vile" in both sub-collections.

In [24]:
##compare with Jen, why get twice as much?

def word_prob(word, counter, asum):
    if word in counter:
        return counter[word]/asum
    else:
        return 0

print ("Probability of delicious in positive reviews: " + str(word_prob("delicious", pos_word_counts, positive_tokens)))
print ("Probability of manager in positive reviews: " + str(word_prob("manager", pos_word_counts, positive_tokens)))
print ("Probability of edgy in positive reviews: " + str(word_prob("edgy", pos_word_counts, positive_tokens)))
print ("Probability of vile in positive reviews: " + str(word_prob("vile", pos_word_counts, positive_tokens)))

print ("Probability of delicious in negative reviews: " + str(word_prob("delicious", neg_word_counts, negative_tokens)))
print ("Probability of manager in positive reviews: " + str(word_prob("manager", neg_word_counts, negative_tokens)))
print ("Probability of edgy in positive reviews: " + str(word_prob("edgy", neg_word_counts, negative_tokens)))
print ("Probability of vile in positive reviews: " + str(word_prob("vile", neg_word_counts, negative_tokens)))
    

Probability of delicious in positive reviews: 0.0012819741921518166
Probability of manager in positive reviews: 0.00016836434043269636
Probability of edgy in positive reviews: 4.810409726648467e-06
Probability of vile in positive reviews: 0
Probability of delicious in negative reviews: 5.539815445005461e-05
Probability of manager in positive reviews: 0.001044650912486744
Probability of edgy in positive reviews: 0
Probability of vile in positive reviews: 7.914022064293515e-06


### Q8: probability of a sequence (version 1.0)

Now create a function `review_prob` that takes a string containing multiple words (for example "the food was delicious"), a counter, and a sum. It should use the same regular expression method you used when reading the files earlier to break this string into tokens, and then use `word_prob` to calculate the probability of each of those word tokens. It should then return the product of those probabilities.

Print the probability of "I loved the carnitas", "but then the manager came out and told us", and "the ambience was edgy but the food was vile" according to both sub-collections. What do you notice about these six probabilities?

In [54]:
import re
def review_prob(sentence, counter, asum):
    p = 1
    my_reg_exp=re.compile("\w+")
    word_list = my_reg_exp.findall(sentence)
    for word in word_list:
        p = p * word_prob(word, counter, asum)
    return p
    
print("Probability of I \"loved the carnitas\" in positive reviews: " + str(review_prob("I loved the carnitas", pos_word_counts, positive_tokens)))
print(review_prob("I loved the carnitas", neg_word_counts, negative_tokens))

print(review_prob("but then the manager came out and told us", pos_word_counts, positive_tokens))
print(review_prob("but then the manager came out and told us", neg_word_counts, negative_tokens))

print(review_prob("the ambience was edgy but the food was vile", pos_word_counts, positive_tokens))
print(review_prob("the ambience was edgy but the food was vile", neg_word_counts, negative_tokens))
    

Probability of I "loved the carnitas" in positive reviews: 1.2254483237140511e-11
5.113217890059061e-13
9.508351338741093e-25
2.4727715407612935e-22
0.0
0.0


Write your observations about the six probabilities here:

This script identifies the probability of a group of words appearing together; essentially the probability of a sentence appearing in a review. "I loved the carnitas" seemed to appear relatively frequently in the positive reviews and an exponentially small amount of times in the negative reviews.

The sentence "the ambience was edgy but the food was vile" does not seem to appear anywhere in the positive or the negative reviews. Searching in the positive.txt and negative.txt files for these sentences reveals that this is in fact the case.

### Q9: calculate word probability (version 2.0)

Instead of multiplying probabilities, we can also add *log* probabilities. Create a function `word_log_prob` that calculates the log probability of a word, and a function `review_log_prob` that calculates the sum of the log probabilities of the words in a string. These should take the same arguments as the previous functions.

Print the log probabilities of the same example words and sentences from the previous problems. Explain any errors that occur in comments.

In [26]:
import math
def word_log_prob(word, counter, asum):
    return math.log((word_prob(word, counter, asum)))
    
    ##if word in counter:
        ##return math.log(counter[word]/asum)
    ##else:
        ##return 0

print (word_log_prob("delicious", pos_word_counts, positive_tokens))
print (word_log_prob("manager", pos_word_counts, positive_tokens))
print (word_log_prob("edgy", pos_word_counts, positive_tokens))
##print (word_log_prob("vile", pos_word_counts, positive_tokens))

print (word_log_prob("delicious", neg_word_counts, negative_tokens))
print (word_log_prob("manager", neg_word_counts, negative_tokens))
##print (word_log_prob("edgy", neg_word_counts, negative_tokens))
print (word_log_prob("vile", neg_word_counts, negative_tokens))

import re
def review_log_prob(sentence, counter, asum):
    p_log = 0
    my_reg_exp=re.compile("\w+")
    word_list = my_reg_exp.findall(sentence)
    for word in word_list:
        prob_log = word_log_prob(word, counter, asum)
        p_log = p_log + prob_log
    return p_log

    
print(review_log_prob("I loved the carnitas", pos_word_counts, positive_tokens))
print(review_log_prob("I loved the carnitas", neg_word_counts, negative_tokens))

print(review_log_prob("but then the manager came out and told us", pos_word_counts, positive_tokens))
print(review_log_prob("but then the manager came out and told us", neg_word_counts, negative_tokens))

##print(review_log_prob("the ambience was edgy but the food was vile", pos_word_counts, positive_tokens))
##print(review_log_prob("the ambience was edgy but the food was vile", neg_word_counts, negative_tokens))

-6.659354051613109
-8.689380233729594
-12.244728295219009
-9.800964277941842
-6.864072504410784
-11.746874426997156
-25.125494875196893
-27.905956014622877
-55.313279441784466
-49.832361245136


If a word type never occurs in our negative reviews, it will have probability zero. But we don't want to rule out the possibility that any future negative review could include that word type. We need to a way to give non-zero probability in both positive and negative polarities to a word that we have only seen in one. A simple way to do this is to add 1 to the count when we calculate word probability, as long as the word has appeared in at least one sub-collection (words we have never seen in either sub-collection will still have probability zero).

This change avoids log-zero errors, but it violates the laws of probability. Why?

Answer here:

By assigning 1 as the probability of a word appearing, you can find the correct probability if it does in fact appear. However, if it does not actually appear, the probability stays at 1 which violates the laws of probability because you are saying that there is a 100% chance of the word appearing which is not in fact the case. 

### Q10: Collection statistics

Create a `Counter` called `both_counts` by adding the two negative and positive counters together (just use +, Counters are awesome). Create a new variable `both_tokens` to represent the total number of tokens in the collection. Create a variable `vocabulary_size` whose value is the total number of distinct word types in the collection. Print the vocabulary size.

In [38]:
both_counts = pos_word_counts + neg_word_counts
both_tokens = 0
for key in both_counts:
    both_tokens = both_tokens + both_counts[key]
print(both_tokens)

vocabulary_size = 0
for key in both_counts:
    vocabulary_size += 1
print(vocabulary_size)

569603
24773


### Q11: Calculate word probability (version 3.0)

Now create a function `smoothed_word_log_prob` that takes the same arguments as `word_prob` (word, counter, sum of the counter), but adds 1 to the count for the word (don't modify the counter, just add 1 in your function). Previously we divided the word count by the sum of all the counts in the counter, which guaranteed that the sum of the probability of all word types was 1.0. Now that we are adding 1 to each word, what should the denominator be so that this new probability distribution will sum to 1.0?

Use this function to print the log probability of the words "delicious", "manager", "edgy", and "vile" in both sub-collections. (These function calls should have the exact same arguments as the previous blocks.)

In [39]:
import math
def smoothed_word_log_prob(word, counter, asum):
    v3count = 0
    v3count = counter[word] + 1
    denominator = asum + len(counter)
    return math.log(v3count/denominator)

    
print (smoothed_word_log_prob("delicious", pos_word_counts, positive_tokens))
print (smoothed_word_log_prob("manager", pos_word_counts, positive_tokens))
print (smoothed_word_log_prob("edgy", pos_word_counts, positive_tokens))
print (smoothed_word_log_prob("vile", pos_word_counts, positive_tokens))

print (smoothed_word_log_prob("delicious", neg_word_counts, negative_tokens))
print (smoothed_word_log_prob("manager", neg_word_counts, negative_tokens))
print (smoothed_word_log_prob("edgy", neg_word_counts, negative_tokens))
print (smoothed_word_log_prob("vile", neg_word_counts, negative_tokens))

-6.706404368872308
-8.724120330791187
-11.888187919164393
-12.986800207832502
-9.714950720874876
-6.875872257366262
-12.017535813868921
-11.324388633308976


### Q12: Test your classifier!

Now let's put it all together. Given a review, we want to know whether that review is more likely to be positive or negative. To answer this question we will calculate the log of the ratio between $P(positive | words\ in\ review)$ and $P(negative | words\ in\ review)$. This ratio is simpler than the two individual. Why?

answer here:


Calculate this ratio for the review "the ambience was edgy but the food was disgusting". Use the probability that a review is positive or negative that you calculated earlier as the prior probabilities $P(positive)$ and $P(negative)$, but this time use the log of those probabilities. Print the resulting log probability ratio.

In [57]:
##P(𝑝𝑜𝑠𝑖𝑡𝑖𝑣𝑒|𝑤𝑜𝑟𝑑𝑠 𝑖𝑛 𝑟𝑒𝑣𝑖𝑒𝑤) = (P(words in review|positive) * p (positive))/P(words in review)
##𝑃(𝑛𝑒𝑔𝑎𝑡𝑖𝑣𝑒|𝑤𝑜𝑟𝑑𝑠 𝑖𝑛 𝑟𝑒𝑣𝑖𝑒𝑤) = (P(words in review|negative) * p (negative))/P(words in review)

def classifier(sentence, pos_counter, neg_counter, pos_sum, neg_sum):
    import math
    p_pos = math.log(prob_pos)
    p_neg = math.log(num_neg_reviews/all_reviews)
    
    my_reg_exp=re.compile("\w+")
    word_list = my_reg_exp.findall(sentence)
    
    p_words_pos = 0
    for word in word_list:
        p_words_pos += smoothed_word_log_prob(word, pos_counter, pos_sum)
        
    p_words_neg = 0
    for word in word_list:
        p_words_neg += smoothed_word_log_prob(word, neg_counter, neg_sum)
    
    p_words = p_words_pos + p_words_neg

    p_pos_words = (p_words_pos*p_pos)/p_words
    p_neg_words = (p_words_neg*p_neg)/p_words

    ratio = p_pos_words/p_neg_words
    return ratio
    
print (classifier("the ambience was edgy but the food was disgusting", pos_word_counts, neg_word_counts, positive_tokens, negative_tokens))



0.13991940336558795


* How should you interpret the log probability ratio?
* Which category is more likely?
* Does the prior probability matter?
* Suggest a prior probability that would change our decision, and explain why.

Answers here: